In [298]:
import pymysql
import pandas as pd
import numpy as np
import urllib.request, json

In [299]:
#grab the entire table from sql
#contains recent traded option log and updated gex/vex/ddoi values
#make this take an argument in case we have more tables in the future
def get_sql_df():
    Host = 'deribit-db-instance.csduabs5w5go.us-east-1.rds.amazonaws.com'
    User = 'admin'
    Password = 'Deribit123'
    Database = 'deribit'
    con = pymysql.connect(user=User, password=Password,host=Host, database=Database, port=3306, ssl_key='../../Downloads/deribit-key-pair.pem')
    try:
        with con.cursor() as cur:
            query = 'SELECT * FROM deribit.option_table'
            cur.execute(query)
            df = pd.read_sql(query, con)
            return df
    finally:
        con.close()

In [300]:
df = get_sql_df()
df

,MyUnknownColumn,trade_seq,trade_id,timestamp,tick_direction,price,mark_price,iv,instrument_name,index_price,direction,amount,gex,vex,ddoi
0,0,3,64622797,1613793025351,0,0.1625,0.245046,107.69,BTC-26MAR21-54000-C,55753.42,buy,10,10,10,0
1,0,2,64649826,1613851501155,0,0.1340,0.073299,165.31,BTC-5MAR21-57000-P,56738.42,buy,3,20,15,0
2,0,50,64649954,1613851546122,0,0.1370,0.063862,198.70,BTC-5MAR21-57000-C,56731.02,buy,3,30,16,0
3,0,57,64650074,1613851603336,0,0.0695,0.061493,134.11,BTC-5MAR21-60000-C,56796.98,buy,3,40,20,0
4,0,336,64657047,1613855054463,2,0.0560,0.151207,39.36,BTC-26FEB21-54000-C,56715.78,sell,2,30,10,0
5,0,2,64657489,1613857287961,2,0.0100,0.003001,114.51,BTC-21FEB21-56000-P,56704.36,buy,1,10,20,0


In [301]:
df.shape

(6, 15)

In [302]:
new_trade = df.iloc[-1]
new_trade
#call get_sql_df to get dataframe of all options
#call get_instrument on each instrument_name to see if its active or not
#create new dataframe based on that
#add new_trade to new dataframe
#call get_order_book on the new dataframe (since it has only active options)
#calculate new gamma/vex/ddoi
#MAYBE: remove any old sql stuff and store them?
#post new trade info + gamma/vex/ddoi

MyUnknownColumn                      0
trade_seq                            2
trade_id                      64657489
timestamp                1613857287961
tick_direction                       2
price                             0.01
mark_price                  0.00300078
iv                              114.51
instrument_name    BTC-21FEB21-56000-P
index_price                    56704.4
direction                          buy
amount                               1
gex                                 10
vex                                 20
ddoi                                 0
Name: 5, dtype: object

In [303]:
testingdf = df.copy()
testingdf

,MyUnknownColumn,trade_seq,trade_id,timestamp,tick_direction,price,mark_price,iv,instrument_name,index_price,direction,amount,gex,vex,ddoi
0,0,3,64622797,1613793025351,0,0.1625,0.245046,107.69,BTC-26MAR21-54000-C,55753.42,buy,10,10,10,0
1,0,2,64649826,1613851501155,0,0.1340,0.073299,165.31,BTC-5MAR21-57000-P,56738.42,buy,3,20,15,0
2,0,50,64649954,1613851546122,0,0.1370,0.063862,198.70,BTC-5MAR21-57000-C,56731.02,buy,3,30,16,0
3,0,57,64650074,1613851603336,0,0.0695,0.061493,134.11,BTC-5MAR21-60000-C,56796.98,buy,3,40,20,0
4,0,336,64657047,1613855054463,2,0.0560,0.151207,39.36,BTC-26FEB21-54000-C,56715.78,sell,2,30,10,0
5,0,2,64657489,1613857287961,2,0.0100,0.003001,114.51,BTC-21FEB21-56000-P,56704.36,buy,1,10,20,0


In [304]:
#working function here
def get_instrument(all_trades):
    for x in all_trades['instrument_name']:
        print("x: ", x)
        url = 'https://test.deribit.com/api/v2/public/get_instrument?instrument_name='
        url += x
        with urllib.request.urlopen(url) as url:
            data = json.loads(url.read().decode())
        result = data['result']
        if(result['is_active']):
            pass
        else:
            iname = result['instrument_name']
            print("iname: " , iname)
            inst_names = all_trades[all_trades['instrument_name'] == iname].index
            all_trades.drop(inst_names, inplace=True)
            #all_trades.drop(testingdf[testingdf['instrument_name'] == iname].index, inplace = True)
    active_options = all_trades
    return active_options

In [305]:
no_stale = get_instrument(testingdf)

x:  BTC-26MAR21-54000-C
x:  BTC-5MAR21-57000-P
x:  BTC-5MAR21-57000-C
x:  BTC-5MAR21-60000-C
x:  BTC-26FEB21-54000-C
iname:  BTC-26FEB21-54000-C
x:  BTC-21FEB21-56000-P
iname:  BTC-21FEB21-56000-P


In [306]:
no_stale

,MyUnknownColumn,trade_seq,trade_id,timestamp,tick_direction,price,mark_price,iv,instrument_name,index_price,direction,amount,gex,vex,ddoi
0,0,3,64622797,1613793025351,0,0.1625,0.245046,107.69,BTC-26MAR21-54000-C,55753.42,buy,10,10,10,0
1,0,2,64649826,1613851501155,0,0.1340,0.073299,165.31,BTC-5MAR21-57000-P,56738.42,buy,3,20,15,0
2,0,50,64649954,1613851546122,0,0.1370,0.063862,198.70,BTC-5MAR21-57000-C,56731.02,buy,3,30,16,0
3,0,57,64650074,1613851603336,0,0.0695,0.061493,134.11,BTC-5MAR21-60000-C,56796.98,buy,3,40,20,0


In [361]:
get_orderbook_url = "https://test.deribit.com/api/v2/public/get_order_book?depth=5&instrument_name=BTC-5MAR21-57000-C"
with urllib.request.urlopen(get_orderbook_url) as get_orderbook_url:
    ob_data = json.loads(get_orderbook_url.read().decode())
ob_data = pd.DataFrame.from_dict(ob_data['result'], orient='index')
ob_data = ob_data.transpose()
ob_data

,underlying_price,underlying_index,timestamp,stats,state,settlement_price,open_interest,min_price,max_price,mark_price,...,estimated_delivery_price,change_id,bids,bid_iv,best_bid_price,best_bid_amount,best_ask_price,best_ask_amount,asks,ask_iv
0,49635.5,SYN.BTC-5MAR21,1614651603482,"{'volume': 3.5, 'price_change': -12.5, 'low': ...",open,0.01,32.6,0.0001,0.203,0.00325438,...,49521.7,5397633915,[],0,0,0,0.0275,3,"[[0.0275, 3.0]]",201.27


In [373]:
def get_orderbook_data(df_to_append_to, instrument_name):
    get_orderbook_url = "https://test.deribit.com/api/v2/public/get_order_book?depth=5&instrument_name="
    get_orderbook_url += instrument_name
    with urllib.request.urlopen(get_orderbook_url) as get_orderbook_url:
        orderbook_data = json.loads(get_orderbook_url.read().decode())
    orderbook_data = pd.DataFrame.from_dict(orderbook_data['result'], orient='index')
    orderbook_data = orderbook_data.transpose()
    combined_df = df_to_append_to.append(orderbook_data, ignore_index=True)
    return combined_df

In [374]:
greeks = pd.DataFrame()
for i in no_stale['instrument_name']:
    greeks = get_orderbook_data(greeks, i)
greeks.head(n=1)

,underlying_price,underlying_index,timestamp,stats,state,settlement_price,open_interest,min_price,max_price,mark_price,...,estimated_delivery_price,change_id,bids,bid_iv,best_bid_price,best_bid_amount,best_ask_price,best_ask_amount,asks,ask_iv
0,50153.9,BTC-26MAR21,1614652007294,"{'volume': 2.0, 'price_change': -76.4423, 'low...",open,0.13,160.1,0.0001,0.3745,0.174949,...,49397.2,5397667437,[],0,0,0,0.3715,30,"[[0.3715, 30.0]]",400.13


In [375]:
#cols = greeks.loc['open_interest', 'mark_iv', 'greeks']
greeks = greeks.loc[:, ['open_interest', 'mark_iv', 'greeks']]
greeks = pd.DataFrame(greeks['greeks'].values.tolist())
greeks

,vega,theta,rho,gamma,delta
0,51.21682,-211.45117,12.33843,0.00002,0.54555
1,6.49140,-100.60332,-4.73706,0.00003,-0.92650
2,6.49459,-100.61850,0.30920,0.00003,0.07354
3,6.38924,-132.57820,0.29859,0.00002,0.07198


In [379]:
total_gamma = no_stale['amount'] * greeks['gamma']
gex = sum(gamma_per_trade)